<a href="https://colab.research.google.com/github/Group-ProjectE17/PCA-Analysics/blob/main/Flex_Robot_Modeling_2_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup

##Initilizations

In [1]:
import numpy as np
from itertools import combinations
from datetime import datetime, timedelta
#import datefinder
import pandas as pd
import string
import os
import os.path
import uuid
import random
from pprint import pprint
import json
import csv
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from numpy import linalg as lin
from sklearn import datasets
import sklearn
from sklearn.mixture import GaussianMixture
import smtplib
from sklearn.model_selection import train_test_split
import sklearn
from sklearn import linear_model
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score
import numpy as np
from itertools import combinations
from datetime import datetime, timedelta
#import datefinder
import pandas as pd
import string
import os
import os.path
import uuid
import random
from pprint import pprint
import json
import csv
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from numpy import linalg as lin
from sklearn import datasets
import sklearn
from sklearn.mixture import GaussianMixture
import smtplib
from sklearn.mixture import GaussianMixture
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import r_regression
from sklearn.preprocessing import PolynomialFeatures
import plotly.figure_factory as ff


from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Function definitions

###Data analysis by Dr Maithreepaa

In [2]:
class mugas_data_analysis:



  
  
    def principle_component_values(self, modelDF, predictDF, nnPCs):
        tempModelDataPDF=modelDF.copy();
        tempPredictDataPDF=predictDF.copy();
        #nnPCs=min(len(tempModelDataPDF.index.values),len(tempModelDataPDF.columns));
        #nnPCs='mle'
        pca=sklearn.decomposition.PCA(n_components=nnPCs);
        modelnumPCA=pca.fit_transform(tempModelDataPDF);
        PCweights=pca.singular_values_;
        PCAmodelScore=pca.score(tempModelDataPDF);
        PCAmeans=pca.mean_
        predictnumPCA=pca.transform(tempPredictDataPDF);
        PCApredictScore=pca.score(tempPredictDataPDF);
        #scaling=sklearn.preprocessing.StandardScaler().fit(pd.DataFrame(numPCA));
        #scaledPCA=scaling.transform(numPCA);
        transformer = sklearn.decomposition.FactorAnalysis(n_components=nnPCs); # n_components=nnPCs, random_state=0);
        modelsvdFactor = transformer.fit_transform(tempModelDataPDF);
        FactorSVDmeans=transformer.mean_
        svdFactormodelScore=transformer.score(tempModelDataPDF);
        predictsvdFactor = transformer.transform(tempPredictDataPDF);
        svdFactorpredictScore=transformer.score(tempPredictDataPDF);
    

        traceNames=[];
        for nn in range(len(PCweights)):
            traceNames+=['P'+str(nn)];

        mindex=tempModelDataPDF.index.values;
        pindex=tempPredictDataPDF.index.values;
        #figFactor=self.hist_plot(PPFactor.T,histBinSize, binRange, traceNames,'Distributions of the PC Factors','PC Factors','Frequency')
        pcaOutput={'PCAmeans':PCAmeans,'FactorSVDmeans':FactorSVDmeans,'traceNames':traceNames, 'fitScores': {'PCApredictScore':PCApredictScore, 'PCAmodelScore':PCAmodelScore, 'svdFactormodelScore':svdFactormodelScore, 'svdFactorpredictScore':svdFactorpredictScore}, 'PCweights':PCweights, 'modelnumPCA':pd.DataFrame(modelnumPCA,index=mindex),'predictnumPCA':pd.DataFrame(predictnumPCA,index=pindex), 'modelsvdFactor':pd.DataFrame(modelsvdFactor,index=mindex),  'predictsvdFactor':pd.DataFrame(predictsvdFactor,index=pindex)}
        return pcaOutput

    def gmm_data_model(self, mData, nClusters):
        #mData - numeric centered np array of shape (ndata,nfactors)

        gmm = sklearn.mixture.GaussianMixture(n_components = nClusters)
        gmm.fit(mData)
        labels = gmm.predict(mData)
        clusterIC=[gmm.aic(mData), gmm.bic(mData)]
        print(clusterIC)
        fig1=md.scatter_plot3D(mData[:,0],mData[:,1],mData[:,2],labels)

        return {'gmm':gmm, 'clusterIC':clusterIC}


    def gmm_data_classifier(self, model_DataPDF, predict_DataPDF,nClusters):
        modelDataPDF0=model_DataPDF.copy();
        predictDataPDF0=predict_DataPDF.copy();
        modelData=modelDataPDF0.to_numpy();
        predictData=predictDataPDF0.to_numpy();
        nPCs=modelData.shape[1]
        #print([modelPCA[n].shape,predictPCA[n].shape])
        fitGMM=self.gmm_data_model(modelData,nClusters)
        gmm=fitGMM['gmm'];

        #Plabels=[];
        #for nn in range(nPCs):
        #    Plabels+=['PF'+str(nn)]

        mLabels = gmm.predict(modelData)
        #modelFitScore=gmm.score(modelData);
        modelDist={'Model Mean':gmm.means_, 'Model Covaraince':gmm.covariances_};
        modelSampleFitScore=gmm.score_samples(modelData);
        modelDataPDF0['labels']=mLabels
        modelDataPDF0['Fit Score']=modelSampleFitScore;

        pLabels = gmm.predict(predictData)
        #predictFitScore=gmm.score(predictData);
        predictProbDist=gmm.predict_proba(predictData);
        predictSampleFitScore=gmm.score_samples(predictData);

        predictDataPDF0['labels']=pLabels
        predictDataPDF0['Fit Score']=predictSampleFitScore;
        xx1=[]; yy1=[]; traceNames1=[];
        xx2=[]; yy2=[]; traceNames2=[];

        for kk in set(modelDataPDF0['labels'].to_list()):
            xx1+=[['All Data']];
            yy1+=[[len(modelDataPDF0[modelDataPDF0['labels']==kk])]];
            yy2+=[[len(predictDataPDF0[predictDataPDF0['labels']==kk])]];
            traceNames1+=['GP'+str(kk)]
        fig21=self.bar_plot(xx1,yy1,traceNames1,'Model Clusters','','Numbers','stack')
        fig31=self.bar_plot(xx1,yy2,traceNames1,'Predicted Clusters','','Numbers','stack')
        #print({'modelFitScore':modelFitScore, 'predictFitScore':predictFitScore})
        return {'labeled model data':modelDataPDF0, 'labeled predict data':predictDataPDF0,'fitScore':{'modelFitScore':modelSampleFitScore, 'predictFitScore':predictSampleFitScore},'Probability Model':modelDist}

    def logistic_reg_data_classifier(self, model_XDataPDF, model_YDataPDF, predict_XDataPDF):
        MXDataPDF=model_XDataPDF.copy();
        MYDataPDF=model_YDataPDF.copy();
        PXDataPDF=predict_XDataPDF.copy();
        clf = sklearn.linear_model.LogisticRegression(random_state=0).fit(MXDataPDF.to_numpy(), MYDataPDF.to_numpy());
        mLabels = clf.predict(MXDataPDF.to_numpy())
        modelDist=clf.predict_proba(MXDataPDF.to_numpy());
        modelSampleFitScore=clf.score(MXDataPDF.to_numpy(), MYDataPDF.to_numpy());
        MXDataPDF['labels']=mLabels
        mLabelTypes=list(set(mLabels));
        probLabels=[];
        for mlt in mLabelTypes:
            probLabels+=['p('+str(mlt)+')']
    
        MXDataPDF[probLabels]=modelDist;

        pLabels = clf.predict(PXDataPDF.to_numpy())
        predictDist=clf.predict_proba(PXDataPDF.to_numpy());
        pLabelTypes=set(probLabels);
        PXDataPDF['labels']=pLabels;
        pLabelTypes=list(set(pLabels));
        probLabels=[];
        for mlt in pLabelTypes:
            probLabels+=['p('+str(mlt)+')']

        PXDataPDF[probLabels]=predictDist;
        
        xx1=[]; yy1=[]; traceNames1=[];
        xx2=[]; yy2=[]; traceNames2=[];

        for kk in set(MXDataPDF['labels'].to_list()):
            xx1+=[['All Data']];
            yy1+=[[len(MXDataPDF[MXDataPDF['labels']==kk])]];
            yy2+=[[len(PXDataPDF[PXDataPDF['labels']==kk])]];
            traceNames1+=['GP-'+str(kk)]
        fig21=self.bar_plot(xx1,yy1,traceNames1,'Model Clusters','','Numbers','stack')
        fig31=self.bar_plot(xx1,yy2,traceNames1,'Predicted Clusters','','Numbers','stack')
        
        
        #print({'modelFitScore':modelFitScore, 'predictFitScore':predictFitScore})
        return {'labeled model data':MXDataPDF, 'labeled predict data':PXDataPDF,'Probability Model':{'modelDist':modelDist,'predictDist':predictDist}}


    def hist_plot(self, xx, binSize, binRange, traceNames, titleText, xLabel,yLabel):
        fig = go.Figure()

        for i,x in enumerate(xx):
            fig.add_trace(go.Histogram(
                x=x,
                #histnorm='percent',
                #cumulative_enabled=cumulative,
                name=traceNames[i], # name used in legend and hover labels
                xbins=dict( # bins used for histogram
                    start=binRange[0],
                    end=binRange[1],
                    size=binSize
                ),
                #marker_color='#EB89B5',
                opacity=0.75
            ))

        fig.update_layout(
            title_text=titleText, # title of plot
            xaxis_title_text=xLabel, # xaxis label
            yaxis_title_text=yLabel, # yaxis label
            bargap=0.2, # gap between bars of adjacent location coordinates
            bargroupgap=0.1 # gap between bars of the same location coordinates
        )

        fig.show()
        #fig.write_html("/content/drive/My Drive/LMS Logs/Course_STATS_upto_May/"+titleText+".html")
        return fig

    def scatter_plot3D(self,xx,yy,zz,color):
        fig = go.Figure(data=[go.Scatter3d(x=xx, y=yy, z=zz, 
                                    mode='markers',marker=dict(
          #size=12,
          color=color,                # set color to an array/list of desired values
          #colorscale='Viridis',   # choose a colorscale
          opacity=0.5
        ))])
        fig.show()
        return fig

    def bar_plot(self, xx, yy, traceName, figTitle, xLabel, yLabel, barmode):
        fig = go.Figure()
        for i,x in enumerate(xx):
            fig.add_trace(go.Bar(y=yy[i], x=x, name=traceName[i]))
        
        
        fig.update_layout(
            title=figTitle, xaxis_title=xLabel, yaxis_title=yLabel, barmode=barmode
            )
        fig.show()
        return fig   

    def linear_regression_modeling(self,model_inputs,model_outputs,test_inputs,test_outputs):
      
        regr = linear_model.LinearRegression()
        linregModel=regr.fit(model_inputs, model_outputs)
        predict_model_outputs=linregModel.predict(model_inputs)
        model_r2_score=r2_score(model_outputs, predict_model_outputs)
        
        predict_test_outputs=linregModel.predict(test_inputs)
        predictScore=linregModel.score(test_inputs,test_outputs)
        test_r2_score=r2_score(test_outputs, predict_test_outputs)
        #print(test_PCA_r2_score)
        print('R2 value = {}'.format(predictScore))

        return {'fitParameters':{'predictScore':predictScore,'model_r2_score':model_r2_score,'test_r2_score':test_r2_score},'predict_model_output':predict_model_outputs,'predict_test_output':predict_test_outputs}

In [3]:
md=mugas_data_analysis()

### Plot functions by Dr Maithreepala

In [4]:
def pi_plot_values(dataPDF, plotCol, valueCol, ifAverage, figTitle):
    xx1=[]; yy1=[]; cc1=[]; traceNames1=[];
    for xx in set(dataPDF[plotCol].tolist()):
      temp=dataPDF[dataPDF[plotCol]==xx];
      xx1+=[xx]
      tempxx=temp[temp[plotCol]==xx]
      cc1+=[];
      if valueCol==None:
        yy1+=[len(tempxx)];
      else:
        if ifAverage:
          yy1+=[sum([float(xx) for xx in tempxx[valueCol].tolist()])/(0.0001+len(tempxx))]
        else:
          yy1+=[sum([float(xx) for xx in tempxx[valueCol].tolist()])];
      
    
    fig = go.Figure()
    
    fig.add_trace(go.Pie(labels=xx1, values=yy1, name=plotCol))
        

    fig.update_traces(hole=0.4, hoverinfo='label+value+name', textinfo='percent')
    fig.update_layout(
        title_text=figTitle,
        #annotations=[dict(text='Colombo', x=0.18, y=0.5, font_size=15, showarrow=False), dict(text='Anuradhapura', x=0.84, y=0.5, font_size=15, showarrow=False)])
        )
    #fig.show()
    return fig

def hist_plot(ifPercentage,dataPDF, binRange, binSize, gpCol, valueCol, titleText, xLabel,yLabel):
  xx1=[]; traceNames=[];
  if gpCol==None:
    gpFldList=['Default']
  else:  
    gpFldList=list(set(dataPDF[gpCol].tolist()))
  gpFldList.sort()
  for xx in gpFldList:
    if gpCol==None:
      temp=dataPDF
    else:
      temp=dataPDF[dataPDF[gpCol]==xx];    
    xx1+=[temp[valueCol].tolist()]
    traceNames+=[xx]
  
  
  fig = go.Figure()
  
  if ifPercentage:
    pltType='percent'
  else:
    pltType=None

  for i,x in enumerate(xx1):
    fig.add_trace(go.Histogram(
        x=x,
        histnorm=pltType,
        #cumulative_enabled=cumulative,
        name=traceNames[i], # name used in legend and hover labels
        xbins=dict( # bins used for histogram
            start=binRange[0],
            end=binRange[1],
            size=binSize
        ),
        #marker_color='#EB89B5',
        opacity=0.75
    ))

  fig.update_layout(
      title_text=titleText, # title of plot
      xaxis_title_text=xLabel, # xaxis label
      yaxis_title_text=yLabel, # yaxis label
      bargap=0.2, # gap between bars of adjacent location coordinates
      bargroupgap=0.1 # gap between bars of the same location coordinates
  )

  #fig.show()
  #fig.write_html("/content/drive/My Drive/LMS Logs/Course_STATS_upto_May/"+titleText+".html")
  return fig

###Plot functions

In [5]:
#Ploting

#Histograms
# ploty
def plot_histograms(data_frame,data_coloum,col='blue'):
  fig = px.histogram(data_frame, x=data_coloum,  title=data_coloum, opacity=0.9,color_discrete_sequence=[col] , nbins=200)
  fig.update(layout_title_text='Data distributions of '+data_coloum)
  fig.show()

#Matplotlib
def plot_histograms_plt(data_frame,data,col="blue"):
  plt.hist(data_frame[data],bins=100 ,color=col, alpha=0.7, density=True)
  plt.title('Data distribution of'+data)
  plt.ylabel("Probability")
  plt.xlabel(data)
  plt.show()

#(2) Dash plots
def plot_DashPlot(data_frame,data_coloum,col='#F66095'):
   hist_data =[data_frame[data_coloum]]
   group_labels = [data_coloum] 
   fig = ff.create_distplot(hist_data, group_labels,bin_size=[0.1],colors=[col])
   fig.show()

#(Box plot)
def plot_BoxPlot(data_frame,data_coloum,col='#F66095'):
   fig = px.box(data_frame, y=data_coloum,points='all')
   fig.update(layout_title_text='Boxplot of '+data_coloum)
   fig.show()

#(Scatter plots)
def plot_ScatterPlot(data_frame,input,output,col='blue'):
   fig = px.scatter(data_frame, x=input, y=output,color_discrete_sequence=[col])
   fig.update(layout_title_text='Variatios of ' +input +' Vs '+output)
   fig.show()
   

###Other functions

In [6]:


def PCA_Analisis(input,output):
    pcaOutputData=md.principle_component_values(output,output,len([*output]))
    pcaInputData=md.principle_component_values(input,input,len([*input]))
    return [ pcaInputData,pcaOutputData]
    
def PCA_invers(pca_val):
    input=pca_val.to_numpy()
    pca = PCA(n_components=4)
    x_reconstructed  = pca.inverse_transform(input )
    return x_reconstructed



def reg_fit2_outputs(Xmodel,ymodel):
    regr = linear_model.LinearRegression()
    yregModel=regr.fit(Xmodel, ymodel)
    yregModelpredict=yregModel.predict(Xmodel)
    print(yregModel.coef_, 'Intercept: '+str(yregModel.intercept_))
    return yregModelpredict

def linear_reg_train(Xmodel,ymodel):
    regr = linear_model.LinearRegression()
    yregModel=regr.fit(Xmodel, ymodel)
    return yregModel

def liner_reg_predict(Xmodel,yregModel):
    yregModelpredict=yregModel.predict(Xmodel)
    return yregModelpredict
     

#Import data And Preprossesing

##Import Data

In [9]:
csvFilePath='/content/drive/My Drive/Project_3rd_year/Data/{}'
columns=['Input1','Input2','Input3','Input4','Output1','Output2','Output3','Output4','Output5']
axial_data_frame=pd.read_csv(csvFilePath.format('UnitCell_Axial_Data_Complete.csv'),header=None)
axial_data_frame.columns=columns
bending_data_frame=pd.read_csv(csvFilePath.format('UnitCell_Bending_Data_Complete.csv'),header=None)
bending_data_frame.columns=columns
twisting_data_frame=pd.read_csv(csvFilePath.format('UnitCell_Twist_Data_Complete.csv'),header=None)
twisting_data_frame.columns=columns

##Data Split

In [10]:
#For Axial data
axial_data_model,axial_data_test=train_test_split(axial_data_frame,test_size=0.2)
#For Bending data
bending_data_model,bending_data_test=train_test_split(bending_data_frame,test_size=0.2)
#For Twist data
twisting_data_model,twisting_data_test=train_test_split(twisting_data_frame,test_size=0.2)


#Axial Data

 There are 4 Inputs and 5 Outputs in the Axial daata frame

* **Input1**- Internal pressure                                  

* **Input2**-X components of the external force

* **Input3** - Y components of the external force

* **Input 4** - Z components of the external force

* **Output1** -Magnitude of the displacement
* **Output2** - X component of the displacement
* **Output3** - Y component of the displacement
* **Output4** -Z components of thr displacemnt
* **Output5** - Resultant force
 


##Functions

In [11]:
# Functions definitions for axial data 
def PCA_Input_output(data):
    outputCols=['Output2','Output3','Output4'] #,'Output5']
    inputCols=['Input1','Input2','Input3','Input4'] #'Input1',
    modelOutputData=data[outputCols]
    predictOutputData=data[outputCols]
    modelInputData=data[inputCols]
    predictInputData=data[inputCols]
    pcaOutputData=md.principle_component_values(modelOutputData,predictOutputData,len([*modelOutputData]))
    pcaInputData=md.principle_component_values(modelInputData,predictInputData,len([*modelInputData]))
    return [pcaOutputData, pcaInputData]

def reg_fit2_outputs(Xmodel,ymodel):
    regr = linear_model.LinearRegression()
    yregModel=regr.fit(Xmodel, ymodel)
    yregModelpredict=yregModel.predict(Xmodel)
    print(yregModel.coef_, 'Intercept: '+str(yregModel.intercept_))
    return yregModelpredict
def axial_unit_cell_model(axial_data_model):
    inputCols=['Input1','Input2','Input4']
    outputCols=['Output2','Output3','Output4']
    model_inputs=axial_data_model[inputCols]
    model_outputs=axial_data_model[outputCols] 
    regr1 = linear_model.LinearRegression()
    linregModel1=regr1.fit(model_inputs, model_outputs)
    model_inputs=axial_data_model[['Input1','Output1']]
    model_outputs=axial_data_model[['Output5']]
    poly = PolynomialFeatures(2)
    model_inputs_poly=pd.DataFrame(poly.fit_transform(model_inputs.to_numpy()))


    regr2 = linear_model.LinearRegression()
    linregModel2=regr2.fit(model_inputs_poly, model_outputs)
    return [linregModel1,linregModel2]

def axial_unit_cell_predict(linregModel1,linregModel2,axial_data_inputs):
    inputCols=['Input1','Input2','Input4']
    axial_data_inputs=axial_data_inputs.copy()
    test_inputs1=axial_data_inputs[inputCols]
    predict_test_outputs1=linregModel1.predict(test_inputs1)
    predict_test_outputs_PDF=pd.DataFrame(predict_test_outputs1,columns=['Output2','Output3','Output4'])
    
    axial_data_inputs['Output1']=(predict_test_outputs_PDF**2).sum(axis='columns').apply(np.sqrt).to_list()
    predict_test_outputs_PDF['Output1']=(predict_test_outputs_PDF**2).sum(axis='columns').apply(np.sqrt).to_list()

    test_inputs2=axial_data_inputs[['Input1','Output1']]
    poly = PolynomialFeatures(2)
    test_inputs2_poly=pd.DataFrame(poly.fit_transform(test_inputs2.to_numpy()))
    predict_test_outputs2=linregModel2.predict(test_inputs2_poly)
    predict_test_outputs_PDF['Output5']=predict_test_outputs2
    return [test_inputs2_poly,predict_test_outputs_PDF]

def axial_unit_cell_model_validation(axial_data_frame):
    axial_data_model,axial_data_test=train_test_split(axial_data_frame,test_size=0.2)
    [linregModel1,linregModel2]=axial_unit_cell_model(axial_data_model)
    inputCols=['Input1','Input2','Input4']
    outputCols=['Output2','Output3','Output4']
    
    test_inputs1=axial_data_test[inputCols].copy()
    test_outputs1=axial_data_test[outputCols]
    predictScore1=linregModel1.score(test_inputs1,test_outputs1)    
    
    [test_inputs2_poly,predict_test_outputs_PDF]=axial_unit_cell_predict(linregModel1,linregModel2,test_inputs1)
    display(predict_test_outputs_PDF)
    predict_test_outputs1=predict_test_outputs_PDF[outputCols].to_numpy()
    predict_test_outputs2=predict_test_outputs_PDF[['Output5']].to_numpy()
    predictScore2=linregModel2.score(test_inputs2_poly,axial_data_test['Output5'])  

    Xreal=test_outputs1
    Xpredict=predict_test_outputs1
    fig1=go.Figure()
    fig1.add_trace(go.Scatter3d(x=Xreal[outputCols[0]],y=Xreal[outputCols[1]].to_list(),z=Xreal[outputCols[2]].to_list(),mode='markers',marker={'size':2},name='Actual'))
    fig1.add_trace(go.Scatter3d(x=Xpredict[:,0],y=Xpredict[:,1],z=Xpredict[:,2],mode='markers',marker={'size':2},name='Predicted'))
    fig1.show()

    Xreal=axial_data_test[['Input1','Output1']]
    Xpredict=predict_test_outputs_PDF[['Output1','Output2','Output3','Output4','Output5']]
    fig2=go.Figure()
    fig2.add_trace(go.Scatter3d(x=Xreal['Input1'],y=Xreal['Output1'],z=axial_data_test['Output5'].to_list(),mode='markers',marker={'size':2},name='Actual'))
    fig2.add_trace(go.Scatter3d(x=Xreal['Input1'],y=Xpredict['Output1'],z=Xpredict['Output5'],mode='markers',marker={'size':2},name='Predicted'))
    fig2.show()


    Xreal=axial_data_test[['Output1','Output2','Output3','Output4','Output5']]
    fig3=go.Figure()
    fig3.add_trace(go.Scatter3d(x=Xreal['Output1'],y=Xreal['Output3'],z=Xreal['Output5'],mode='markers',marker={'size':2},name='Actual'))
    fig3.add_trace(go.Scatter3d(x=Xpredict['Output1'],y=Xpredict['Output3'],z=Xpredict['Output5'],mode='markers',marker={'size':2},name='Predicted'))
    fig3.show()

    r2Score=r2_score(axial_data_test[['Output1','Output2','Output3','Output4','Output5']],predict_test_outputs_PDF[['Output1','Output2','Output3','Output4','Output5']])
    return {'r2Score':r2Score,'predictScore1':predictScore1,'predictScore2':predictScore2}

##Data visualizations

In [16]:
data=axial_data_frame 
data_input=data[['Input1','Input2','Input3','Input4']]
data_Output=data[['Output1','Output2','Output3','Output4']]

### Data distributions

####Inputs

In [ ]:

#Histogram plots of inputs
for label in data_input:
  plot_histograms(data_input,label)


In [ ]:
#Box plots of inputs
for label in data_input:
  plot_BoxPlot(data_input,label)

In [ ]:
#Dash plots plots of inputs
for label in data_input:
   plot_DashPlot(data_input,label)

####Outputs

In [ ]:
#histograms plots of Outputs
for label in data_Output:
  plot_histograms(data_Output,label)

In [ ]:
#Box plots of inputs
for label in data_Output:
  plot_BoxPlot(data_Output,label)

In [ ]:
#Dash plots plots of inputs
for label in data_input:
   plot_DashPlot(data_input,label)

### Input Output 

In [ ]:
# Scatter plots of input vs outputs
for label1 in data_Output :
     plot_ScatterPlot(data_input,'Input2',label1)

##Correlation Analysis

According to the following correlation analysis input output correlations can be approximated as
* **Output 1** - Output2 Output2 Output4 ($Output1=\sqrt{Output2^2+Output3^2+Output4^2}$)
* **Output 2** -Input1 Input2 input4 (Linear appximation)
* **Output 3** -Input1 Input2 input4 (Linear appximation)
* **Output 4** -Input1 Input2 input4 (Linear appximation)
* **Output 5** -Input1 output1 (non liner appximation)


###Input output correlation

In [ ]:
outputLabels=['Output1','Output2','Output3','Output4','Output5']
X=axial_data_frame[['Input1','Input2','Input3','Input4']]
fig=go.Figure()
rvals=[]
for i,lbl in enumerate(outputLabels):
    y=axial_data_frame[lbl]
    rr=r_regression(X, y.to_numpy())
    print(rr)
    fig.add_trace(go.Scatter(y=rr,name=lbl))
fig.show()

###Output-output correlation

In [ ]:
outputLabels=['Output1','Output2','Output3','Output4','Output5']
X=axial_data_frame[outputLabels]
fig=go.Figure()
rvals=[]
for i,lbl in enumerate(outputLabels):
    y=axial_data_frame[lbl]
    rr=r_regression(X, y.to_numpy())
    print(rr)
    fig.add_trace(go.Scatter(y=rr,name=lbl))
fig.show()

###Input Output Relations 

In [ ]:
#Output1 <-----(Output2 ,Output3,Output4)

data=axial_data_frame 
data_input=data[['Input1','Input2','Input3','Input4']]
data_Output=data[['Output1','Output2','Output3','Output4','Output5']]

fig=go.Figure()
fig.add_trace(go.Scatter3d(x=data_Output['Output2'].to_list(),y=data_Output['Output4'].to_list(),z=data_Output['Output1'].to_list(),mode='markers',marker={'size':2},name='Actual'))
fig.show()

In [ ]:
#Output2 /Output 3/Output 4 <-----(input 1, Input 2,input 4)

fig=go.Figure()
fig.add_trace(go.Scatter3d(x=data_input['Input1'].to_list(),y=data_input['Input4'].to_list(),z=data_Output['Output2'].to_list(),mode='markers',marker={'size':2},name='Actual'))
fig.show()

In [ ]:
#Output5 <----- (Input1,Output1)
fig=go.Figure()
fig.add_trace(go.Scatter3d(x=data_input['Input1'].to_list(),y=data_Output['Output1'].to_list(),z=data_Output['Output5'].to_list(),mode='markers',marker={'size':2},name='Actual'))
fig.show()

##Modeing and Validation

###Liner Regression -All Outputs with All Inputs

In [ ]:
model_inputs=axial_data_model[['Input1','Input2','Input3','Input4']]
model_outputs=axial_data_model[['Output1','Output2','Output3','Output4','Output5']] 
test_inputs=axial_data_test[['Input1','Input2','Input3','Input4']]
test_outputs=axial_data_test[['Output1','Output2','Output3','Output4','Output5']] #
lin_reg_model=md.linear_regression_modeling(model_inputs,model_outputs,test_inputs,test_outputs)
Xreal=test_outputs
Xpredict=lin_reg_model['predict_test_output']
fig=go.Figure()
fig.add_trace(go.Scatter3d(x=Xreal['Output2'],y=Xreal['Output3'].to_list(),z=Xreal['Output4'].to_list(),mode='markers',marker={'size':2},name='Actual'))
fig.add_trace(go.Scatter3d(x=Xpredict[:,1],y=Xpredict[:,2],z=Xpredict[:,3],mode='markers',marker={'size':2},name='Predicted'))
fig.show()

### Linear Regression -Output2 Output3 Output4 with Input1 Input2 input4

In [ ]:
inputCols=['Input1','Input2','Input4']
outputCols=['Output2','Output3','Output4']
model_inputs=axial_data_model[inputCols]
model_outputs=axial_data_model[outputCols] 
test_inputs=axial_data_test[inputCols]
test_outputs=axial_data_test[outputCols] #
lin_reg_model=md.linear_regression_modeling(model_inputs,model_outputs,test_inputs,test_outputs)
Xreal=test_outputs
Xpredict=lin_reg_model['predict_test_output']
fig=go.Figure()
fig.add_trace(go.Scatter3d(x=Xreal[outputCols[0]],y=Xreal[outputCols[1]].to_list(),z=Xreal[outputCols[2]].to_list(),mode='markers',marker={'size':2},name='Actual'))
fig.add_trace(go.Scatter3d(x=Xpredict[:,0],y=Xpredict[:,1],z=Xpredict[:,2],mode='markers',marker={'size':2},name='Predicted'))
fig.show()

In [ ]:
(pd.DataFrame(Xpredict)**2).sum(axis='columns').apply(np.sqrt)

###Liner regression -Output5 with Input1 Output1

In [ ]:
model_inputs=axial_data_model[['Input1','Output1']]
model_outputs=axial_data_model[['Output5']] 
test_inputs=axial_data_test[['Input1','Output1']]
test_outputs=axial_data_test[['Output5']] #
lin_reg_model=md.linear_regression_modeling(model_inputs,model_outputs,test_inputs,test_outputs)
Xreal=test_inputs
Xpredict=lin_reg_model['predict_test_output']
fig=go.Figure()
fig.add_trace(go.Scatter3d(x=Xreal['Input1'],y=Xreal['Output1'],z=test_outputs['Output5'].to_list(),mode='markers',marker={'size':2},name='Actual'))
fig.add_trace(go.Scatter3d(x=Xreal['Input1'],y=Xreal['Output1'],z=Xpredict[:,0],mode='markers',marker={'size':2},name='Predicted'))
fig.show()

###Liner regression -Output5 with Input1 Output1(Polynomial approximation of Inputs)

In [ ]:
model_inputs=axial_data_model[['Input1','Output1']]
model_outputs=axial_data_model[['Output5']] 
test_inputs=axial_data_test[['Input1','Output1']]
test_outputs=axial_data_test[['Output5']] #
poly = PolynomialFeatures(2)
model_inputs_poly=pd.DataFrame(poly.fit_transform(model_inputs.to_numpy()))
test_inputs_poly=pd.DataFrame(poly.fit_transform(test_inputs.to_numpy()))
lin_reg_model=md.linear_regression_modeling(model_inputs_poly,model_outputs,test_inputs_poly,test_outputs)
Xreal=test_inputs
Xpredict=lin_reg_model['predict_test_output']
fig=go.Figure()
fig.add_trace(go.Scatter3d(x=Xreal['Input1'],y=Xreal['Output1'],z=test_outputs['Output5'].to_list(),mode='markers',marker={'size':2},name='Actual'))
fig.add_trace(go.Scatter3d(x=Xreal['Input1'],y=Xreal['Output1'],z=Xpredict[:,0],mode='markers',marker={'size':2},name='Predicted'))
fig.show()

### Model development

In [ ]:
#Prediction
axial_data_model,axial_data_test=train_test_split(axial_data_frame,test_size=0.2)
[linregModel1,linregModel2]=axial_unit_cell_model(axial_data_model)
axial_unit_cell_predict(linregModel1,linregModel2,axial_data_test[['Input1','Input2','Input4']])

In [ ]:
#validation
axial_data_model,axial_data_test=train_test_split(axial_data_frame,test_size=0.2)
outputs=axial_unit_cell_model_validation(axial_data_frame)

#Bending data

#Twisting data